In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import random
import sys
sys.path.append("../scripts/")

In [2]:
from bert4rec_model import RecommendationModel

In [3]:
import pandas as pd

In [4]:
DATA_PATH = "../data/"

In [5]:
ratings_df = pd.read_csv(f"{DATA_PATH}ratings_mapped.csv")

In [6]:
ratings_df.head()

,userId,movieId,rating,timestamp,movieId_mapped
0,429,595,5.0,828124615,514
1,429,588,5.0,828124615,508
2,429,590,5.0,828124615,510
3,429,592,5.0,828124615,511
4,429,432,3.0,828124615,378


In [7]:
ratings_df.movieId.nunique()

9724

In [8]:
movies_df = pd.read_csv(f"{DATA_PATH}movies_mapped.csv")

In [9]:
movies_df.head()

,movieId,title,genres,movieId_mapped
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,2.0
1,2,Jumanji (1995),Adventure|Children|Fantasy,3.0
2,3,Grumpier Old Men (1995),Comedy|Romance,4.0
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,5.0
4,5,Father of the Bride Part II (1995),Comedy,6.0


In [10]:
movies_df.title.nunique()

9737

In [11]:
movies_df[movies_df.title == "Star Wars: Episode V - The Empire Strikes Back (1980)"]

,movieId,title,genres,movieId_mapped
898,1196,Star Wars: Episode V - The Empire Strikes Back...,Action|Adventure|Sci-Fi,899.0


In [12]:
movies_df[movies_df.title == "Star Wars: Episode VI - Return of the Jedi (1983)"]

,movieId,title,genres,movieId_mapped
911,1210,Star Wars: Episode VI - Return of the Jedi (1983),Action|Adventure|Sci-Fi,912.0


In [51]:
movies_df[movies_df.title == "Star Wars: Episode IV - A New Hope (1977)"]

,movieId,title,genres,movieId_mapped
224,260,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Sci-Fi,226.0


In [52]:
model = RecommendationModel(vocab_size=9726, heads = 4, layers=6, emb_dim=512, pad_id=0, num_pos=120)

NUM POS:  120


In [53]:
model.encoder.word_embedding

Embedding(9726, 512, padding_idx=0)

In [54]:
new_word_embedding = nn.Embedding(59048, 512, 0)

In [55]:
model.encoder.word_embedding.weight.size()

torch.Size([9726, 512])

In [56]:
model.encoder.word_embedding.weight.is_leaf

True

In [57]:
new_word_embedding.weight.requires_grad = False

In [58]:
new_word_embedding.weight.requires_grad

False

In [61]:
model.encoder.word_embedding.weight.clone().detach()

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-1.7665,  1.6926,  0.4056,  ..., -0.9145, -0.6684, -2.7191],
        [-0.4989, -1.2712, -1.2487,  ...,  1.2529, -0.2603,  0.3139],
        ...,
        [ 0.8683,  1.6797, -1.0878,  ...,  0.2916,  0.3808,  0.2060],
        [-0.2326,  0.3657, -0.7110,  ...,  0.4804, -1.9811,  0.3621],
        [-0.5933, -0.1471, -2.0153,  ...,  1.5211,  1.2693,  1.7062]])

In [62]:

new_word_embedding.weight[:model.encoder.word_embedding.weight.size(0)] = model.encoder.word_embedding.weight.clone().detach()

In [30]:
torch.Tensor(model.encoder.word_embedding.weight)

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 1.8485,  0.1677, -0.7663,  ...,  0.2392, -1.9132, -0.8887],
        [-0.8623, -0.2828, -0.5505,  ..., -0.2717,  1.5624, -1.2634],
        ...,
        [-0.0023,  1.6211,  0.0222,  ...,  0.4915,  0.2782,  0.0946],
        [-0.0690, -0.6950,  0.5203,  ...,  1.2444,  0.5209, -0.8068],
        [-0.4467, -0.3993,  0.2656,  ...,  0.1667, -1.5695, -1.1303]],
       grad_fn=<SliceBackward>)

In [63]:
new_word_embedding.weight

Parameter containing:
tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-1.7665,  1.6926,  0.4056,  ..., -0.9145, -0.6684, -2.7191],
        [-0.4989, -1.2712, -1.2487,  ...,  1.2529, -0.2603,  0.3139],
        ...,
        [ 0.4575,  0.4710,  0.1874,  ...,  0.5162,  1.1350, -0.4312],
        [ 2.0730, -0.6797,  1.3500,  ...,  0.9539,  0.2212, -1.0174],
        [-0.7637,  0.4035,  2.3838,  ...,  0.7317,  0.2953, -0.2601]])

In [64]:
new_word_embedding.weight.is_leaf

True

In [32]:
new_word_embedding.weight.requires_grad = True

RuntimeError: you can only change requires_grad flags of leaf variables.

In [34]:
new_word_embedding.requires_grad_()

Embedding(59048, 512, padding_idx=0)

In [37]:
new_word_embedding.requires_grad_(True)

Embedding(59048, 512, padding_idx=0)

In [39]:
new_word_embedding.weight.requires_grad

True

In [65]:
model.encoder.word_embedding.weight.is_leaf

True

In [66]:
model.encoder.word_embedding = new_word_embedding

In [67]:
model.encoder.word_embedding

Embedding(59048, 512, padding_idx=0)

In [42]:
model.encoder.word_embedding.weight.requires_grad

True

In [68]:
model.encoder.word_embedding.weight.is_leaf

True

In [45]:
model.encoder.word_embedding.weight.is_leaf = True

AttributeError: attribute 'is_leaf' of 'torch._C._TensorBase' objects is not writable

In [43]:
model.encoder.word_embedding.weight.retain_grad()

In [39]:
sd = torch.load("../models/rec-10/model_files/bert4rec-state-dict.pth", map_location = "cpu")["state_dict"]

In [45]:
type(model)

bert4rec_model.RecommendationModel

In [46]:
model.load_state_dict(sd)

<All keys matched successfully>

In [47]:
model = model.eval()

In [48]:
model(torch.LongTensor([899, 912, 226]))

RuntimeError: The size of tensor a (3) must match the size of tensor b (120) at non-singleton dimension 1

In [49]:
t = torch.LongTensor([899, 912, 226])

In [50]:
zerso = torch.zeros((120))

In [51]:
zerso[:t.size(0)] = t

In [54]:
t = zerso.long()

In [55]:
t

tensor([899, 912, 226,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0])

In [57]:
op = model(t)

In [58]:
op.size()

torch.Size([120, 1, 9726])

In [59]:
_, pred = op.max(2)

In [61]:
pred.flatten()

tensor([ 899,  912,  226, 5816,   62,   62,  961,  512,  512,  512,  512,  512,
         512,  512,  512, 6995, 6519,  512,  512,  512,  512,  512,  620, 1577,
         512,  512,  509,  512,  512,  512,  735, 1055,   99,  961,  512,  512,
        1544, 6995, 4356, 4356, 6995, 6995,  512,  512, 3138, 3138, 3138, 4793,
         512,  118,  512, 3138,   99,   99,   99,  512,  512,  620, 6995, 6995,
        6995, 6995, 1796, 3851, 3851, 3851, 3851,  512, 3138,  795,  795,  795,
        3138, 3138, 3138, 3138, 3138, 3138, 3138, 3138, 3138, 5365, 3559, 1961,
        1961, 1961, 1961, 1961, 1961, 1961, 1961, 1961, 1961, 1961, 1084, 1084,
        1084, 1084,  942,  942,  942,  942,  942,  942,  942,  942,  942, 1084,
        1084, 1084,   34, 2672, 2672,  689, 1053, 1053, 7754,  118,  118,   62])

In [71]:
op.size()

torch.Size([120, 1, 9726])

In [ ]:
trg = [899, 912, 226]
while not len(trg) > 20:
    zerso = torch.zeros((120))
    zerso[:trg.size(0)] = torch.Tensor(trg)
    t = zerso.long()
    op = model(t)
    _, pred = op.max(2)

In [62]:
movies_df[movies_df.movieId_mapped == 5816]

,movieId,title,genres,movieId_mapped
5829,32289,Ice Princess (2005),Children|Comedy|Drama,5816.0


In [63]:
movies_df[movies_df.movieId_mapped == 62]

,movieId,title,genres,movieId_mapped
60,68,French Twist (Gazon maudit) (1995),Comedy|Romance,62.0


In [64]:
movies_df[movies_df.movieId_mapped == 961]

,movieId,title,genres,movieId_mapped
960,1261,Evil Dead II (Dead by Dawn) (1987),Action|Comedy|Fantasy|Horror,961.0


In [65]:
movies_df[movies_df.movieId_mapped == 512]

,movieId,title,genres,movieId_mapped
510,593,"Silence of the Lambs, The (1991)",Crime|Horror|Thriller,512.0


In [67]:
movies_df[movies_df.movieId_mapped == 6995]

,movieId,title,genres,movieId_mapped
7010,68157,Inglourious Basterds (2009),Action|Drama|War,6995.0


In [ ]:
movies_df[movies_df.movieId_mapped == 6995]